In [4]:
#Install and Import Required Modules,  LLMs and Agent Tools
!pip -q install langchain huggingface_hub openai google-search-results

import os

os.environ["OPENAI_API_KEY"] = ""  # https://platform.openai.com

os.environ['SERPAPI_API_KEY'] = "" # https://serpapi.com/

In [5]:
# @title
import os

os.environ["OPENAI_API_KEY"] = ""  # https://platform.openai.com
os.environ['SERPAPI_API_KEY'] = "" # https://serpapi.com/

#Out of the box agent
## LangChain Zero-Shot ReAct Agent


In [6]:
# import libraries

from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

Now we can configure the LLM, the tools we will use, and the agent that allows us to leverage the ReAct framework together with the LLM and tools. Note that we are using a search API for searching external information and LLM as a math tool.



In [8]:
llm = OpenAI(temperature=0) # First, let's load the language model we're going to use to control the agent.

In [9]:
tools = load_tools(["serpapi"], llm=llm) #Next, let's load some tools to use.

In [10]:
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True) #Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.

Once that's configured, we can now run the agent with the desired query/prompt.

In [11]:
agent.run("Who is Olivia Wilde's boyfriend? What is his current age raised to the 0.23 power?")




> Entering new AgentExecutor chain...
 I should search for information about Olivia Wilde and her boyfriend.
Action: Search
Action Input: Olivia Wilde boyfriend
Observation: ["Looks like Olivia Wilde and Jason Sudeikis are starting 2023 on good terms. Amid their highly publicized custody battle – and the actress' ..."]
Thought: I should search for Jason Sudeikis' age.
Action: Search
Action Input: Jason Sudeikis age
Observation: 48 years
Thought: I now know the final answer
Final Answer: Jason Sudeikis' age raised to the 0.23 power is approximately 2.7.

> Finished chain.


"Jason Sudeikis' age raised to the 0.23 power is approximately 2.7."

In [12]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain import OpenAI, SerpAPIWrapper, LLMChain

search = SerpAPIWrapper()
tools = [
   Tool(
       name="Search",
       func=search.run,
       description="useful for when you need to answer questions about current events",
   )
]




#Custom agent
## LangChain Zero-Shot ReAct Agent

In [13]:

prefix = """Answer the following questions as best you can, but do it in old Shakepearean English. You have access to the following tools:"""
suffix = """Begin! Remember to speak in old Shakepearean English in the final answer. Use the words "behold" at least once.

Question: {input}
{agent_scratchpad}"""




In [14]:
prompt = ZeroShotAgent.create_prompt(
   tools, prefix=prefix, suffix=suffix, input_variables=["input", "agent_scratchpad"]
)

In [16]:
print(prompt.template)


Answer the following questions as best you can, but do it in old Shakepearean English. You have access to the following tools:

Search: useful for when you need to answer questions about current events

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to speak in old Shakepearean English in the final answer. Use the words "behold" at least once.

Question: {input}
{agent_scratchpad}


Note that we are able to feed agents a self-defined prompt template, i.e. not restricted to the prompt generated by the create_prompt function, assuming it meets the agent's requirements.

For example, for ZeroShotAgent, we will need to ensure that it meets the following requirements. There should a string starting with "Action:" and a following string starting with "Action Input:", and both should be separated by a newline.



In [17]:
llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)



In [18]:
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)


In [19]:

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

In [20]:
agent_executor.run("How many hurricanes are expected to make landfall in the US this year?")





> Entering new AgentExecutor chain...
Thought: I must find out how many hurricanes are expected to make landfall in the US this year.
Action: Search
Action Input: Number of hurricanes expected to make landfall in the US this year
Observation: ['NOAA is forecasting a range of 12 to 17 total named storms (winds of 39 mph or higher). Of those, 5 to 9 could become hurricanes (winds of 74 ...']
Thought: I now know the final answer
Final Answer: Behold, 'twould seem that the range of hurricanes expected to make landfall in the US this year is from five to nine.

> Finished chain.


"Behold, 'twould seem that the range of hurricanes expected to make landfall in the US this year is from five to nine."